In [1]:
import tensorflow as tf
from tensorflow import keras
from sklearn import model_selection as ms
from sklearn import preprocessing as pp
import numpy as np

print('1')
print(tf.__version__)
np.set_printoptions(suppress=True)

def tryLabelEncoder(df):
    for column in df.columns:
       if df[column].dtype == type(object):
            le = pp.LabelEncoder()
            df[column] = le.fit_transform(df[column])

/Users/dominic/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1
1.10.0


In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing as pp

# Train Data
train = 'train.csv'
test = 'test.csv'
na  = 'missing'
#'n_samples','n_features','n_classes','n_clusters_per_class','n_informative','flip_y','scale' 
#'penalty','l1_ratio','alpha','max_iter','random_state','n_jobs'
train_data = pd.read_csv(train,na_values='none')
train_data = pd.get_dummies(train_data)
test_data = pd.read_csv(test,na_values='none')
test_data = pd.get_dummies(test_data)
print(train_data.head())
# print (train_data.head())
# print (test_data.head())
temp = train_data.fillna(na)
# tryLabelEncoder(temp)
X_train = temp.drop(['time','id'],axis=1).values
Y_train = train_data['time'].values
Y_train = Y_train.ravel()

# Test Data
ID = test_data['id'].values
temp = test_data.fillna(na)
# tryLabelEncoder(temp)
TEST = temp.drop(['id'],axis=1).values
# print(X_train)
# print(Y_train)

   id  l1_ratio   alpha  max_iter  random_state  n_jobs  n_samples  \
0   0  0.304083  0.0001       417           475      -1       1089   
1   1  0.727744  0.0010       578           569       1        790   
2   2  0.745885  0.0100       588           529       2        428   
3   3  0.474605  0.0010       829           103       4        877   
4   4  0.395049  0.0010       167           418       2        216   

   n_features  n_classes  n_clusters_per_class  n_informative    flip_y  \
0         327          4                     3              7  0.074798   
1         373          4                     5              7  0.077781   
2        1198          2                     5              6  0.030196   
3         313          6                     5              7  0.057261   
4         644          8                     5             11  0.073728   

       scale      time  penalty_elasticnet  penalty_l1  penalty_l2  
0  24.242009  0.409987                   0           0     

   id  l1_ratio   alpha  max_iter  random_state  n_jobs  n_samples  \
0   0  0.304083  0.0001       417           475      -1       1089   
1   1  0.727744  0.0010       578           569       1        790   
2   2  0.745885  0.0100       588           529       2        428   
3   3  0.474605  0.0010       829           103       4        877   
4   4  0.395049  0.0010       167           418       2        216   

   n_features  n_classes  n_clusters_per_class  n_informative    flip_y  \
0         327          4                     3              7  0.074798   
1         373          4                     5              7  0.077781   
2        1198          2                     5              6  0.030196   
3         313          6                     5              7  0.057261   
4         644          8                     5             11  0.073728   

       scale      time  penalty_elasticnet  penalty_l1  penalty_l2  
0  24.242009  0.409987                   0           0     

In [3]:
# Step0. Pre-processing
# 调用fit方法，根据已有的训练数据创建一个标准化的转换器

# penalty = temp['penalty'].unique()
# rule = []
# i = 0
# for p in penalty:
#     rule.append([p,i])
#     i += 1
# print(rule)

# for r in X_train:
#     if r[0] == rule[0][0]: r[0] = rule[0][1]
#     elif r[0] == rule[1][0]: r[0] = rule[1][1]
#     elif r[0] == rule[2][0]: r[0] = rule[2][1]
#     elif r[0] == rule[3][0]: r[0] = rule[3][1]
 
scaler = pp.StandardScaler().fit(X_train)
# scaler = pp.RobustScaler(with_centering=False).fit(X_train,with_centering=False)
print(scaler)
# X_train = scaler.transform(X_train)
# print(X_train)

X = X_train
Y = Y_train

# Spliting Training Data, Training Test Data and Validation Data
X_train,X_tmp,Y_train,Y_tmp = ms.train_test_split(X,Y,test_size = 0.2,random_state =13)
X_test,X_val,Y_test,Y_val = ms.train_test_split(X_tmp,Y_tmp,test_size = 0.2,random_state =13)

# X is train data X, Y is train data Y

# for r in TEST:
#     if r[0] == rule[0][0]: r[0] = rule[0][1]
#     elif r[0] == rule[1][0]: r[0] = rule[1][1]
#     elif r[0] == rule[2][0]: r[0] = rule[2][1]
#     elif r[0] == rule[3][0]: r[0] = rule[3][1]
        
TEST = scaler.transform(TEST)
# TEST is TEST DATA X
print(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)
[[  0.43786681   0.0001     997.         ...   0.           0.
    0.        ]
 [  0.81140953   0.01       638.         ...   0.           0.
    1.        ]
 [  0.12520034   0.0001     711.         ...   0.           1.
    0.        ]
 ...
 [  0.81609605   0.001      251.         ...   0.           0.
    1.        ]
 [  0.38643597   0.01       202.         ...   0.           1.
    0.        ]
 [  0.26836232   0.001      938.         ...   0.           0.
    1.        ]]


In [12]:
# ##################################################
# ##################################################
#######XGBOOST Model#######################################

from sklearn.model_selection import GridSearchCV  #grid search
from xgboost.sklearn import XGBRegressor
import xgboost as xgb
import matplotlib.pylab as plt

def adjustParas(param_test,XGB):
    gsearch = GridSearchCV(estimator = XGB, 
                           param_grid = param_test1,scoring='roc_auc',n_jobs=4,iid=False, cv=5) 
    gsearch.fit(X_train,Y_train)
    gsearch.grid_scores_, gsearch.best_params_, gsearch.best_score_

# xgb_param = XGB.get_xgb_params()
# xgtrain = xgb.DMatrix(X_train, label=Y_train)
# cv_folds = 5
# early_stopping_rounds = 50
# cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=XGB.get_params()['n_estimators'], nfold=cv_folds,
#     early_stopping_rounds=early_stopping_rounds)
# print("best 'n_estimators':",cvresult.shape[0])
# XGB.set_params(n_estimators=cvresult.shape[0])


# param_test1 = {
#  'max_depth':range(3,10,2),
#  'min_child_weight':range(1,6,2)
# }
# adjustParas(param_test1,XGB)

# param_test2 = {
#  'max_depth':[4,5,6],
#  'min_child_weight':[4,5,6]
# }
# adjustParas(param_test2,XGB)


# param_test3 = {
#  'gamma':[i/10.0 for i in range(0,5)]
# }
# adjustParas(param_test3,XGB)


# param_test4 = {
#  'subsample':[i/10.0 for i in range(6,10)],
#  'colsample_bytree':[i/10.0 for i in range(6,10)]
# }
# adjustParas(param_test4,XGB)


# param_test5 = {
#  'subsample':[i/100.0 for i in range(75,90,5)],
#  'colsample_bytree':[i/100.0 for i in range(75,90,5)]
# }
# adjustParas(param_test5,XGB)


# param_test6 = {
#  'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
# }
# adjustParas(param_test6,XGB)


# param_test7 = {
#  'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]
# }
# adjustParas(param_test7,XGB)


# print('done')

# XGB = XGBRegressor(learning_rate =0.007,max_depth=6,min_child_weight=0,gamma=0,
#                    subsample=0.8,colsample_bytree=0.8,reg_alpha=0,
#                    n_estimators=1000,objective= 'reg:linear',nthread=4,
#                    scale_pos_weight=1,seed=30,target = 'Disbursed',IDcol = 'ID')
                   
# XGB.fit(X_train,Y_train,eval_metric='auc')
# predictions = XGB.predict(X_test)
# mse = ((Y_test - predictions) ** 2).mean(axis=0)
# print ("MSE : %.4g" % mse)


# pre = XGB.predict(TEST)
# print(pre)

##Another test:
params = {'min_child_weight':[0,1,2,3], 'gamma':[i/10.0 for i in range(0,6)],  'subsample':[0.8],
'colsample_bytree':[0.8], 'max_depth': [2,3,4],'learning_rate':[0.0005,0.00001,0.00005,0.00001]}

# Initialize XGB and GridSearch
xgb = XGBRegressor(nthread=-1) 

grid = GridSearchCV(xgb, params)
grid.fit(X_train, Y_train)

from sklearn.metrics import r2_score

# Print the r2 score
print(r2_score(Y_val, grid.best_estimator_.predict(X_val))) 

pre = grid.best_estimator_.predict(TEST)

-0.5905982918682047


MSE : 1.297
[ 0.10115477  1.6192045  -0.12277973 -0.11765504  0.30448192  1.5818298
 -0.12177831  0.09265903  0.14085537 -0.12177831  0.00807503  0.88607216
 -0.1254934   0.45554304  0.99332356  0.75144434  0.7909187   0.30041486
  1.646663    0.2981587   0.7577449  -0.144813    0.02835214  0.67775154
  0.5358447  -0.296708    1.0278287   0.4790691   0.51109993 -0.21702987
  0.51109993  0.56017035 -0.28779054  0.6562432   0.97090256  1.6996697
  1.0693197   0.32946062  0.23822221 -0.09618938  1.1442604   0.72158027
  0.33639616 -0.10604578  0.47710085  0.72158027  1.1261207  -0.12781495
 -0.14993781  0.08631095 -0.14627069  1.2378552  -0.12409997  1.6478484
  0.01437554 -0.22557217  0.40738678  0.1182439   0.47179604  0.4762948
 -0.1254934   1.8169764  -0.25097477  0.17390159  1.783718    0.7232448
  0.4208628  -0.10092098  1.5125655   0.00807503 -0.3089673  -0.1254934
  0.75403976  0.56096697  1.7077149   0.00807503  0.03465265 -0.29042578
  1.6478484  -0.14993781  0.14924872  0.75774

In [5]:
# # ##################################################
# # ##################################################
# #######Model#######################################
# # # Data from Tensorflow Toturials

# # # mnist = tf.keras.datasets.mnist

x_train = X_train
y_train = Y_train

x_test = X_test
y_test = Y_test

# (x_train, y_train),(x_test, y_test) = x_train,y_train
# x_train, x_test = x_train / 255.0, x_test / 255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
#   tf.keras.layers.Dropout(0.2),
#   tf.keras.layers.Dense(42, activation=tf.nn.softmax)
    tf.keras.layers.Dense(42, activation=tf.nn.softmax)

])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=1000)
model.evaluate(x_test, y_test)
print(model)
pre = model.predict(TEST)
print(pre)
# # ##################################################
# # ##################################################

Epoch 1/1000
320/320 [==============================] - 1s 2ms/step - loss: 11.4135 - acc: 0.0000e+00
Epoch 2/1000
320/320 [==============================] - 0s 406us/step - loss: 10.3257 - acc: 0.0000e+00
Epoch 3/1000
320/320 [==============================] - 0s 346us/step - loss: 10.3257 - acc: 0.0000e+00
Epoch 4/1000
320/320 [==============================] - 0s 353us/step - loss: 10.3257 - acc: 0.0000e+00
Epoch 5/1000
320/320 [==============================] - 0s 344us/step - loss: 10.3257 - acc: 0.0000e+00
Epoch 6/1000
320/320 [==============================] - 0s 345us/step - loss: 10.3257 - acc: 0.0000e+00
Epoch 7/1000
320/320 [==============================] - 0s 334us/step - loss: 10.3257 - acc: 0.0000e+00
Epoch 8/1000
320/320 [==============================] - 0s 357us/step - loss: 10.3257 - acc: 0.0000e+00
Epoch 9/1000
320/320 [==============================] - 0s 373us/step - loss: 10.3257 - acc: 0.0000e+00
Epoch 10/1000
320/320 [==============================] - 0s 333us/

KeyboardInterrupt: 

In [131]:
feature_cols = ['penalty','l1_ratio','alpha','max_iter',
                'random_state','n_jobs','n_samples','n_features',
                'n_classes','n_clusters_per_class','n_informative',
                'flip_y','scale']

regressor = tf.estimator.DNNRegressor(feature_columns=feature_cols,
                                      hidden_units=[50,50],
                                     model_dir='tf_wx_model')
def wx_input_fn(X,y=None,num_epochs=None,shuffle=True,batch_size=20):
    return tf.estimator.inputs.pandas_input_fn(x=X,y=y,
                                               num_epochs=num_epochs,
                                              shuffle=shuffle,
                                              batch_size=batch_size)

evaluations = []
STEPS = 400
for i in range(100):
    regressor.train(input_fn=wx_input_fn(X_train,y=Y_train),steps=STEPS)
    evaluations.append(regressor.evaluate(input_fn=wx_input_fn(X_val,
                                                               y_val,
                                                               num_epochs=1,
                                                               shuffle=False)))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'tf_wx_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a2b3843c8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


/Users/dominic/anaconda3/lib/python3.6/site-packages/tensorflow/python/estimator/inputs/pandas_io.py:106: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if target_column in x:


AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [9]:
# pre = model.predict(TEST)
# Step3. Output file
with open ('xgbtraintest.csv','w') as file:
    file.write('Id,Time\n')
    for i in range(0,len(pre)):
        file.write(str(ID[i]))
        file.write(',')
        if pre[i] < 0:
            file.write('0')
            file.write('\n')
        else:
            file.write(str(pre[i]))
            file.write('\n')



In [148]:
a = 4

In [149]:
a

4